In [102]:
import shutil
from glob import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import librosa
from librosa import feature
import re
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn import metrics
import itertools
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from imblearn.over_sampling import SMOTE 

In [3]:
### Code seperate the wav file and txt file
# src = r'F:\dataset\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files'
# dest = r'F:\dataset\Respiratory_Sound_Database\Respiratory_Sound_Database\files'
# extension = '.wav'
# for root, subfolders, filenames in os.walk(src):
# #     print(root)
# #     print(subfolders)
# #     print(filenames)
#     for file in filenames:
#         if(file.endswith(extension)):
#             path = os.path.join(root, file)
#             shutil.move(path,dest)

In [4]:
data_dir = "D:\DATASET\Speech_Datasets\Respiratory_Sound_Database\Respiratory_Sound_Database\files"
audio_files = glob(os.path.join(data_dir , '*.wav'))
print(len(audio_files))

920


In [5]:
fn_list_i = [
 feature.chroma_stft,
 feature.spectral_centroid,
 feature.spectral_bandwidth,
 feature.spectral_rolloff]
 
fn_list_ii = [
 feature.rms,
 feature.zero_crossing_rate]


pattern = re.compile(r'\d\d\d')

def get_feature_vector(file,y,sr): 
    id1 = []
    a = pattern.search(file)
    id1.append(a.group())
    feat_vect_i = [np.mean(funct(y,sr)) for funct in fn_list_i]
    feat_vect_ii = [np.mean(funct(y)) for funct in fn_list_ii] 
    feature_vector = id1 + feat_vect_i + feat_vect_ii #list concatanation
    return feature_vector


In [64]:
norm_audios_feat = []
for file in audio_files:    
    y,sr = librosa.load(file, sr=None)    #y = time-series array , sr = sampling rate
    feature_vector = get_feature_vector(file, y, sr)
    norm_audios_feat.append(feature_vector)

E:\Anaconda3\lib\site-packages\librosa\core\pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")


In [67]:
import csv

audio_file = 'C:\Users\Krina\Audio Processing\respirtaory_disease_detection_using_ML\respirtaory_features.csv'
header = ['id','chroma_stft', 'spectral_centroid','spectral_bandwidth','spectral_rolloff','rmse','zero_crossing_rate']

with open (audio_file,'w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(norm_audios_feat)


In [75]:
mfcc_vector = []
for file in audio_files:
    id1 = []
    file_name = file.split("\\")[-1]
    id1.append(file_name.split("_")[0])
    y,sr = librosa.load(file,sr=16000)
    mfcc = librosa.feature.mfcc(y,sr,n_mfcc=13).T.tolist()
    mfcc_1 = np.mean(mfcc, axis=0).tolist()
    mfcc_vector.append(id1 + mfcc_1)

In [76]:
import csv

norm_output = 'C:\Users\Krina\Audio Processing\respirtaory_disease_detection_using_ML\mfcc_features.csv'
header = ['id']
for i in range(1,14):
    header.append('mfcc - ' + str(i))

with open (norm_output,'w') as f:
    csv_writer = csv.writer(f, delimiter = ',')
    csv_writer.writerow(header)
    csv_writer.writerows(mfcc_vector)

In [77]:
mfcc = pd.read_csv('C:\Users\Krina\Audio Processing\respirtaory_disease_detection_using_ML\mfcc_features.csv')
audio = pd.read_csv('C:\Users\Krina\Audio Processing\respirtaory_disease_detection_using_ML\respirtaory_features.csv')
# df.head()
# print(df.shape)

In [78]:
mfcc.head()

,id,mfcc - 1,mfcc - 2,mfcc - 3,mfcc - 4,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13
0,101,-515.698943,113.960590,71.166173,42.369430,33.501165,28.330211,20.508287,16.753931,18.437936,16.671231,9.041621,3.930885,6.298694
1,101,-572.717931,103.288583,56.094221,30.831492,33.563887,37.270963,28.716998,18.998847,16.929647,15.496640,9.766257,6.060370,8.856569
2,102,-580.988766,121.589029,56.972991,25.953187,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239
3,103,-408.989858,71.455142,48.836396,34.063806,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760
4,104,-436.699122,116.139438,77.727287,38.066508,12.890244,4.524391,6.382153,11.373524,15.818668,18.493562,18.937647,17.166859,13.774422


In [79]:
final_Df = audio.merge(mfcc, on='id')
final_Df.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc - 1,mfcc - 2,mfcc - 3,mfcc - 4,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13
0,101,0.790317,100.769410,421.074708,100.326977,0.057676,0.002898,-515.698943,113.960590,71.166173,42.369430,33.501165,28.330211,20.508287,16.753931,18.437936,16.671231,9.041621,3.930885,6.298694
1,101,0.790317,100.769410,421.074708,100.326977,0.057676,0.002898,-572.717931,103.288583,56.094221,30.831492,33.563887,37.270963,28.716998,18.998847,16.929647,15.496640,9.766257,6.060370,8.856569
2,101,0.786649,111.913011,535.275912,75.903554,0.034545,0.002503,-515.698943,113.960590,71.166173,42.369430,33.501165,28.330211,20.508287,16.753931,18.437936,16.671231,9.041621,3.930885,6.298694
3,101,0.786649,111.913011,535.275912,75.903554,0.034545,0.002503,-572.717931,103.288583,56.094221,30.831492,33.563887,37.270963,28.716998,18.998847,16.929647,15.496640,9.766257,6.060370,8.856569
4,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014,-580.988766,121.589029,56.972991,25.953187,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239


In [80]:
final_Df = final_Df.groupby('id',as_index=False).agg('mean')
final_Df.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc - 1,mfcc - 2,mfcc - 3,mfcc - 4,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13
0,101,0.788483,106.341210,478.175310,88.115266,0.046111,0.002700,-544.208437,108.624587,63.630197,36.600461,33.532526,32.800587,24.612642,17.876389,17.683792,16.083936,9.403939,4.995627,7.577631
1,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014,-580.988766,121.589029,56.972991,25.953187,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239
2,103,0.776724,104.833348,562.750815,38.039137,0.255904,0.001465,-408.989858,71.455142,48.836396,34.063806,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760
3,104,0.627749,108.450674,153.273541,210.582248,0.058936,0.007269,-439.710849,97.601193,69.050068,39.048891,18.613349,9.490852,7.909424,10.076911,13.689739,16.820580,17.742319,15.963111,12.430860
4,105,0.579455,792.488395,971.752301,1256.439696,0.029107,0.069132,-339.230309,190.384630,-15.817387,-46.475974,17.833985,19.237400,4.010118,1.151154,1.593777,3.363842,7.783992,11.781842,2.682762


In [81]:
#merge label files
df1 = pd.read_csv('F:\dataset\Respiratory_Sound_Database\Respiratory_Sound_Database\patient_diagnosis.csv',header=None)
df1.columns = ['id','label']
df1.head()

,id,label
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


In [84]:
df = final_Df.merge(df1, on='id')
df.head()

,id,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,rmse,zero_crossing_rate,mfcc - 1,mfcc - 2,mfcc - 3,...,mfcc - 5,mfcc - 6,mfcc - 7,mfcc - 8,mfcc - 9,mfcc - 10,mfcc - 11,mfcc - 12,mfcc - 13,label
0,101,0.788483,106.341210,478.175310,88.115266,0.046111,0.002700,-544.208437,108.624587,63.630197,...,33.532526,32.800587,24.612642,17.876389,17.683792,16.083936,9.403939,4.995627,7.577631,URTI
1,102,0.776444,143.511891,572.593659,139.376997,0.022517,0.003014,-580.988766,121.589029,56.972991,...,35.002577,43.354305,28.374711,6.873962,-0.104578,5.379598,10.388685,9.226119,4.562239,Healthy
2,103,0.776724,104.833348,562.750815,38.039137,0.255904,0.001465,-408.989858,71.455142,48.836396,...,30.040957,26.992179,21.588698,17.739098,16.417752,14.404756,11.547453,10.237860,10.428760,Asthma
3,104,0.627749,108.450674,153.273541,210.582248,0.058936,0.007269,-439.710849,97.601193,69.050068,...,18.613349,9.490852,7.909424,10.076911,13.689739,16.820580,17.742319,15.963111,12.430860,COPD
4,105,0.579455,792.488395,971.752301,1256.439696,0.029107,0.069132,-339.230309,190.384630,-15.817387,...,17.833985,19.237400,4.010118,1.151154,1.593777,3.363842,7.783992,11.781842,2.682762,URTI


In [85]:
df['label'].unique()

array(['URTI', 'Healthy', 'Asthma', 'COPD', 'LRTI', 'Bronchiectasis',
       'Pneumonia', 'Bronchiolitis'], dtype=object)

In [87]:
X = df.iloc[:,0:20]
y = df.iloc[:,20:]

print(y)

         label
0         URTI
1      Healthy
2       Asthma
3         COPD
4         URTI
..         ...
121       COPD
122       COPD
123    Healthy
124    Healthy
125  Pneumonia

[126 rows x 1 columns]


In [ ]:
# df = df[df['label'] != 'LRTI']
# df = df[df['label'] != 'Asthma']
# print(df)

In [88]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=777)
X_ROS, y_ROS = ros.fit_sample(X, y)

In [89]:
print(y_ROS)

       label
0       URTI
1    Healthy
2     Asthma
3       COPD
4       URTI
..       ...
507     URTI
508     URTI
509     URTI
510     URTI
511     URTI

[512 rows x 1 columns]


In [103]:
one_hot = LabelEncoder()
y_temp_rows = one_hot.fit_transform(y_ROS)
y_temp_rows

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


array([7, 4, 0, 3, 7, 3, 3, 5, 3, 3, 1, 3, 3, 3, 5, 1, 3, 3, 7, 3, 4, 6,
       4, 3, 4, 4, 4, 3, 7, 3, 7, 3, 3, 3, 6, 4, 7, 3, 3, 6, 3, 3, 4, 4,
       3, 3, 3, 7, 2, 7, 3, 4, 4, 3, 3, 3, 3, 3, 4, 3, 2, 3, 3, 7, 7, 3,
       2, 1, 1, 3, 4, 3, 2, 3, 3, 3, 3, 3, 4, 3, 3, 4, 4, 4, 3, 3, 4, 7,
       3, 7, 6, 3, 3, 4, 3, 1, 7, 3, 3, 3, 1, 4, 3, 3, 3, 2, 3, 4, 4, 7,
       3, 3, 3, 4, 1, 2, 4, 3, 6, 3, 3, 3, 3, 4, 4, 6, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [ ]:
#mapping
# res = {}
# for cl in labelencoder.classes_:
#     res.update({cl:labelencoder.transform([cl])[0]})

# print(res)

In [91]:
df['label'].value_counts()  #inbalance dataset

COPD              64
Healthy           26
URTI              14
Bronchiectasis     7
Bronchiolitis      6
Pneumonia          6
LRTI               2
Asthma             1
Name: label, dtype: int64

In [ ]:
# mfcc = []
# for file in audio_files:
#     y, sr = librosa.load(file)
#     current_mfcc = librosa.feature.mfcc(y,sr,n_mfcc=40)
#     mfcc.append(current_mfcc)


In [104]:
x_train,x_test,y_train,y_test = train_test_split(X_ROS,y_temp_rows,test_size=0.20)

In [106]:
# sm = SMOTE(sampling_strategy='minority', random_state=None, k_neighbors=3) 
# X_train_res, y_train_res = sm.fit_sample(x_train, y_train) 


In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion_matrix',normalize= False,cmap= plt.cm.Blues):
    
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
def one_hot_encoder(true_labels, num_records, num_classes):
    temp = np.array(true_labels[:num_records])
    true_labels = np.zeros((num_records, num_classes))
    true_labels[np.arange(num_records), temp] = 1
    return true_labels

def display_results(y_test, pred_probs, cm=True):
    pred = np.argmax(pred_probs, axis=-1)
    one_hot_true = one_hot_encoder(y_test, len(pred), len(disease_dict))
    print('Test Set Accuracy =  {0:.3f}'.format(accuracy_score(y_test, pred)))
    print('Test Set F-score =  {0:.3f}'.format(f1_score(y_test, pred, average='macro')))
    print('Test Set Precision =  {0:.3f}'.format(precision_score(y_test, pred, average='macro')))
    print('Test Set Recall =  {0:.3f}'.format(recall_score(y_test, pred, average='macro')))
    if cm:
        plot_confusion_matrix(confusion_matrix(y_test, pred), classes=disease_keys)

In [117]:
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(x_train, y_train)

# Predict
pred_probs = rf_classifier.predict(x_test)
accuracy_score(y_test,pred_probs)
# display_results(y_test_ROS, pred_probs)


0.9514563106796117

In [118]:
y_test

array([6, 1, 7, 0, 3, 5, 3, 4, 6, 2, 3, 7, 3, 0, 6, 6, 4, 2, 1, 0, 6, 0,
       5, 4, 2, 0, 0, 2, 0, 2, 6, 2, 0, 1, 3, 0, 6, 2, 0, 6, 7, 4, 6, 1,
       0, 7, 4, 6, 5, 7, 0, 0, 2, 1, 1, 2, 5, 6, 3, 2, 3, 2, 5, 5, 6, 5,
       2, 2, 2, 7, 5, 6, 3, 0, 0, 3, 6, 2, 3, 1, 1, 3, 4, 1, 6, 7, 4, 7,
       3, 3, 0, 1, 7, 2, 4, 4, 3, 2, 4, 1, 1, 3, 5])

In [119]:
pred_probs

array([6, 1, 7, 0, 3, 5, 3, 4, 6, 2, 3, 4, 3, 0, 6, 6, 4, 2, 1, 0, 6, 0,
       5, 4, 2, 0, 0, 2, 0, 2, 6, 2, 0, 1, 3, 0, 6, 2, 0, 6, 7, 4, 6, 1,
       0, 4, 4, 6, 5, 7, 0, 0, 2, 1, 1, 2, 5, 6, 3, 2, 3, 2, 5, 5, 6, 5,
       2, 2, 2, 7, 5, 6, 3, 0, 0, 4, 6, 2, 3, 1, 1, 5, 4, 1, 6, 7, 4, 4,
       3, 3, 0, 1, 7, 2, 4, 4, 3, 2, 4, 1, 1, 3, 5])

In [120]:
from sklearn import metrics
print(metrics.classification_report(y_test, pred_probs, target_names=df['label'].unique()))

                precision    recall  f1-score   support

          URTI       1.00      1.00      1.00        16
       Healthy       1.00      1.00      1.00        12
        Asthma       1.00      1.00      1.00        17
          COPD       1.00      0.87      0.93        15
          LRTI       0.71      1.00      0.83        10
Bronchiectasis       0.90      1.00      0.95         9
     Pneumonia       1.00      1.00      1.00        15
 Bronchiolitis       1.00      0.67      0.80         9

      accuracy                           0.95       103
     macro avg       0.95      0.94      0.94       103
  weighted avg       0.96      0.95      0.95       103



In [113]:
df[df['id']==111]['label']

10    Bronchiectasis
Name: label, dtype: object

In [ ]:
df['label'].unique()